Generalized Get Max Timestamp Notebook
=======================================
Retrieves maximum timestamp from any table/column configuration.

Prerequsite: The lakehouse which holds the targeted table must be set as default lakehouse for this notebook

Parameter: 
- table_name: Table name (required)
- date_column: Date/datetime column name (required) 
- time_column: Optional separate time column (default: None)
- output_format: Output format (default: "%Y-%m-%d %H:%M:%S")

In [8]:
# =============================================================================
# PARAMETERS
# =============================================================================
table_name = "Facts"
date_column = "Changed"
time_column = "None"
output_format = "%Y-%m-%d %H:%M:%S"

StatementMeta(, 6d7fc7d0-8d40-4554-9dd3-af3ae9925a37, 10, Finished, Available, Finished)

In [9]:
# =============================================================================
# IMPORTS & SPARK CONFIG
# =============================================================================
from pyspark.sql import functions as F
from notebookutils import mssparkutils
from datetime import datetime, date

# Parquet datetime handling
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInWrite", "CORRECTED")
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInWrite", "CORRECTED")

StatementMeta(, 6d7fc7d0-8d40-4554-9dd3-af3ae9925a37, 11, Finished, Available, Finished)

In [10]:
# =============================================================================
# MAIN LOGIC
# =============================================================================
def get_max_timestamp(table_name, date_column, time_column, output_format):

    # Validate inputs
    if not table_name or not date_column:
        raise ValueError("table_name and date_column are required")
    
    # Load table
    try:
        df = spark.read.table(table_name)
    except Exception as e:
        raise RuntimeError(f"Failed to load table '{table_name}': {str(e)}")
    
    # Return empty string if table is empty
    if df.rdd.isEmpty():
        return ""
    
    # Process based on column configuration
    if time_column != "None":
        # Separate date + time columns
        if date_column not in df.columns or time_column not in df.columns:
            raise ValueError(
                f"Required columns not found. "
                f"Needed: [{date_column}, {time_column}]. "
                f"Available: {df.columns}"
            )
        
        max_ts = (
            df.withColumn(
                "_ts",
                F.to_timestamp(
                    F.concat_ws(" ",
                        F.date_format(F.col(date_column), "yyyy-MM-dd"),
                        F.col(time_column).cast("string")
                    ),
                    "yyyy-MM-dd HH:mm:ss"
                )
            )
            .select(F.max("_ts"))
            .collect()[0][0]
        )
    else:
        # Single datetime/date column
        if date_column not in df.columns:
            raise ValueError(
                f"Column '{date_column}' not found. "
                f"Available: {df.columns}"
            )
        
        max_ts = df.select(F.max(date_column)).collect()[0][0]
    
    # Format and return
    if max_ts is None:
        return ""
    elif isinstance(max_ts, datetime):
        return max_ts.strftime(output_format)
    elif isinstance(max_ts, date):
        return datetime.combine(max_ts, datetime.min.time()).strftime(output_format)
    else:
        return str(max_ts)

StatementMeta(, 6d7fc7d0-8d40-4554-9dd3-af3ae9925a37, 12, Finished, Available, Finished)

In [11]:
# =============================================================================
# EXECUTION
# =============================================================================
latest_timestamp_str = get_max_timestamp(table_name, date_column, time_column, output_format)

StatementMeta(, 6d7fc7d0-8d40-4554-9dd3-af3ae9925a37, 13, Finished, Available, Finished)

In [12]:
# =============================================================================
# RETURN TO PIPELINE
# =============================================================================
mssparkutils.notebook.exit(latest_timestamp_str)

StatementMeta(, 6d7fc7d0-8d40-4554-9dd3-af3ae9925a37, 14, Finished, Available, Finished)

ExitValue: 2025-11-10 07:26:11

### One cell version

In [13]:
# =============================================================================
# PARAMETERS
# =============================================================================
table_name = "Facts"
date_column = "Changed"
time_column = "None"
output_format = "%Y-%m-%d %H:%M:%S"

# =============================================================================
# IMPORTS & SPARK CONFIG
# =============================================================================
from pyspark.sql import functions as F
from notebookutils import mssparkutils
from datetime import datetime, date

# Parquet datetime handling
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInWrite", "CORRECTED")
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInWrite", "CORRECTED")

# =============================================================================
# MAIN LOGIC
# =============================================================================
def get_max_timestamp(table_name, date_column, time_column, output_format):

    # Validate inputs
    if not table_name or not date_column:
        raise ValueError("table_name and date_column are required")
    
    # Load table
    try:
        df = spark.read.table(table_name)
    except Exception as e:
        raise RuntimeError(f"Failed to load table '{table_name}': {str(e)}")
    
    # Return empty string if table is empty
    if df.rdd.isEmpty():
        return ""
    
    # Process based on column configuration
    if time_column != "None":
        # Separate date + time columns
        if date_column not in df.columns or time_column not in df.columns:
            raise ValueError(
                f"Required columns not found. "
                f"Needed: [{date_column}, {time_column}]. "
                f"Available: {df.columns}"
            )
        
        max_ts = (
            df.withColumn(
                "_ts",
                F.to_timestamp(
                    F.concat_ws(" ",
                        F.date_format(F.col(date_column), "yyyy-MM-dd"),
                        F.col(time_column).cast("string")
                    ),
                    "yyyy-MM-dd HH:mm:ss"
                )
            )
            .select(F.max("_ts"))
            .collect()[0][0]
        )
    else:
        # Single datetime/date column
        if date_column not in df.columns:
            raise ValueError(
                f"Column '{date_column}' not found. "
                f"Available: {df.columns}"
            )
        
        max_ts = df.select(F.max(date_column)).collect()[0][0]
    
    # Format and return
    if max_ts is None:
        return ""
    elif isinstance(max_ts, datetime):
        return max_ts.strftime(output_format)
    elif isinstance(max_ts, date):
        return datetime.combine(max_ts, datetime.min.time()).strftime(output_format)
    else:
        return str(max_ts)

# =============================================================================
# EXECUTION
# =============================================================================
latest_timestamp_str = get_max_timestamp(table_name, date_column, time_column, output_format)

# =============================================================================
# RETURN TO PIPELINE
# =============================================================================
mssparkutils.notebook.exit(latest_timestamp_str)

StatementMeta(, 6d7fc7d0-8d40-4554-9dd3-af3ae9925a37, 15, Finished, Available, Finished)

ExitValue: 2025-11-10 07:26:11